> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
### Escreva sua resposta aqui
import pandas as pd

# Carregar o dataset
url = "https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv?raw=true"
df = pd.read_csv(url)

# Exibir as primeiras linhas do DataFrame para inspeção
print(df.head())

# Exibir informações sobre o DataFrame, como tipo de dados e valores não nulos
print(df.info())

# Exibir estatísticas descritivas das colunas numéricas
print(df.describe())

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import enum
import pandas as pd

# Carregar os dados
url = "https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv?raw=true"
df = pd.read_csv(url)

# Definir a base declarativa
Base = declarative_base()

# Definir os Enums
class SexEnum(enum.Enum):
    F = 'F'
    M = 'M'

class DesignationEnum(enum.Enum):
    Analyst = 'Analyst'
    Senior_Analyst = 'Senior Analyst'
    Manager = 'Manager'
    # Adicione outros valores únicos encontrados na coluna DESIGNATION

class UnitEnum(enum.Enum):
    IT = 'IT'
    Finance = 'Finance'
    Marketing = 'Marketing'
    # Adicione outros valores únicos encontrados na coluna UNIT

# Definir o modelo
class Salary(Base):
    __tablename__ = 'salaries'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

    def __repr__(self):
        return f"<Salary(id={self.id}, first_name='{self.first_name}', last_name='{self.last_name}')>"

# Criar o banco de dados e as tabelas
engine = create_engine('sqlite:///salaries.db')  # Use SQLite para simplicidade
Base.metadata.create_all(engine)

# Criar uma sessão para interagir com o banco de dados
Session = sessionmaker(bind=engine)
session = Session()

# Converter as colunas de data para o tipo datetime
df['DOJ'] = pd.to_datetime(df['DOJ'])
df['CURRENT DATE'] = pd.to_datetime(df['CURRENT DATE'])

# Converter os dados do DataFrame para objetos Salary e adicioná-los à sessão
for index, row in df.iterrows():
    salary = Salary(
        first_name=row['FIRST NAME'],
        last_name=row['LAST NAME'],
        sex=SexEnum(row['SEX']),
        doj=row['DOJ'].to_pydatetime(),
        current_date=row['CURRENT DATE'].to_pydatetime(),
        designation=DesignationEnum(row['DESIGNATION']),
        age=row['AGE'],
        salary=row['SALARY'],
        unit=UnitEnum(row['UNIT']),
        leaves_used=row['LEAVES USED'],
        leaves_remaining=row['LEAVES REMAINING'],
        ratings=row['RATINGS'],
        past_exp=row['PAST EXP']
    )
    session.add(salary)

# Commit e fechar a sessão
session.commit()
session.close()

print("Dados carregados com sucesso no banco de dados!")

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine

# Criar a conexão com o banco de dados SQLite chamado 'salarios.db'
engine = create_engine('sqlite:///salarios.db')

# A variável 'engine' agora representa a conexão com o banco de dados
# Você pode usá-la para executar consultas SQL ou interagir com o ORM

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
### Escreva sua resposta aqui
# from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import sessionmaker
import enum
import pandas as pd

# Carregar os dados
url = "https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv?raw=true"
df = pd.read_csv(url)

# Obter valores únicos para DESIGNATION e UNIT
designation_unique_values = df['DESIGNATION'].unique()
unit_unique_values = df['UNIT'].unique()

# Definir a base declarativa
# Base = declarative_base()

# Definir os Enums
class SexEnum(enum.Enum):
    F = 'F'
    M = 'M'

class DesignationEnum(enum.Enum):
    Analyst = 'Analyst'
    Senior_Analyst = 'Senior Analyst'
    Manager = 'Manager'
    # Adicionar outros valores únicos encontrados na coluna DESIGNATION
    for value in designation_unique_values:
        locals()[value.replace(" ", "_")] = value  # Cria atributos dinamicamente

class UnitEnum(enum.Enum):
    IT = 'IT'
    Finance = 'Finance'
    Marketing = 'Marketing'
    # Adicionar outros valores únicos encontrados na coluna UNIT
    for value in unit_unique_values:
        locals()[value.replace(" ", "_")] = value  # Cria atributos dinamicamente

# Definir o modelo
# class Salary(Base):
#     __tablename__ = 'salaries'

#     id = Column(Integer, primary_key=True, autoincrement=True)
#     first_name = Column(String)
#     last_name = Column(String)
#     sex = Column(Enum(SexEnum))
#     doj = Column(Date)
#     current_date = Column(Date)
#     designation = Column(Enum(DesignationEnum))
#     age = Column(Integer)
#     salary = Column(Float)
#     unit = Column(Enum(UnitEnum))
#     leaves_used = Column(Integer)
#     leaves_remaining = Column(Integer)
#     ratings = Column(Float)
#     past_exp = Column(Float)

#     def __repr__(self):
#         return f"<Salary(id={self.id}, first_name='{self.first_name}', last_name='{self.last_name}')>"

# Criar o banco de dados e as tabelas
# engine = create_engine('sqlite:///salarios.db')  # Use SQLite para simplicidade
# Base.metadata.create_all(engine)

# Criar uma sessão para interagir com o banco de dados
# Session = sessionmaker(bind=engine)
# session = Session()

# Converter as colunas de data para o tipo datetime
df['DOJ'] = pd.to_datetime(df['DOJ'])
df['CURRENT DATE'] = pd.to_datetime(df['CURRENT DATE'])

# Converter os dados do DataFrame para objetos Salary e adicioná-los à sessão
# for index, row in df.iterrows():
#     salary = Salary(
#         first_name=row['FIRST NAME'],
#         last_name=row['LAST NAME'],
#         sex=SexEnum(row['SEX']),
#         doj=row['DOJ'].to_pydatetime(),
#         current_date=row['CURRENT DATE'].to_pydatetime(),
#         designation=DesignationEnum(row['DESIGNATION']),
#         age=row['AGE'],
#         salary=row['SALARY'],
#         unit=UnitEnum(row['UNIT']),
#         leaves_used=row['LEAVES USED'],
#         leaves_remaining=row['LEAVES REMAINING'],
#         ratings=row['RATINGS'],
#         past_exp=row['PAST EXP']
#     )
#     session.add(salary)

# Commit e fechar a sessão
# session.commit()
# session.close()

print("Código para criar as tabelas fornecido. Certifique-se de ter o SQLAlchemy instalado para executá-lo.")

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
### Escreva sua resposta aqui
import pandas as pd
from sqlalchemy import create_engine

# Carregar os dados
url = "https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv?raw=true"
df = pd.read_csv(url)

# Criar a conexão com o banco de dados SQLite chamado 'salarios.db'
engine = create_engine('sqlite:///salarios.db')

# Converter as colunas de data para o formato datetime
df['DOJ'] = pd.to_datetime(df['DOJ'])
df['CURRENT DATE'] = pd.to_datetime(df['CURRENT DATE'])

# Usar o método to_sql para popular o banco de dados
df.to_sql('salaries', engine, if_exists='append', index=False)

print("Dados do CSV carregados com sucesso no banco de dados 'salarios'!")

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
### Execute aqui sua query SQL com SQLAlchemy
with engine.connect() as connection:
#     query = text("""
#         SELECT
#             DESIGNATION,
#             MIN(SALARY / 12) as min_salary,
#             MAX(SALARY / 12) as max_salary,
#             AVG(SALARY / 12) as avg_salary
#         FROM
#             salaries
#         GROUP BY
#             DESIGNATION
#     """)
#     result = connection.execute(query)
#     print("Resultados da query SQL com SQLAlchemy:")
#     for row in result:
#         print(row)

In [ ]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
2. Executando a query SQL com Pandas e SQLAlchemy
# with engine.connect() as connection:
#     query = """
#         SELECT
#             DESIGNATION,
#             MIN(SALARY / 12) as min_salary,
#             MAX(SALARY / 12) as max_salary,
#             AVG(SALARY / 12) as avg_salary
#         FROM
#             salaries
#         GROUP BY
#             DESIGNATION
#     """
#     df_result = pd.read_sql_query(query, connection)
#     print("\nResultados da query SQL com Pandas e SQLAlchemy:")
#     print(df_result)

In [ ]:
### Execute aqui sua query com SQLAlchemy ORM
3. Executando a query com SQLAlchemy ORM
# Session = Session(engine)

# query = select(
#     Salary.designation,
#     func.min(Salary.salary / 12).label('min_salary'),
#     func.max(Salary.salary / 12).label('max_salary'),
#     func.avg(Salary.salary / 12).label('avg_salary')
# ).group_by(Salary.designation)

# result = Session.execute(query)

# print("\nResultados da query com SQLAlchemy ORM:")
# for row in result:
#     print(row)

# Session.close()